# Stroke Prediction

***

Wir verwenden eine **Datenset von Kaggle**: https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset.
Ziel ist es ein **Model** zu erstellen das korrekt anhand der Daten **vorhersagen** kann ob jemand einen sog. *stroke* hatte.

***

Bitte vorher den korrekten Kernel auswaehlen und diesen Command im Terminal nutzen:

`pip install -r requirements.txt`

***

## Importieren der Bibliotheken

***

Zuerst werden natuerlich die verwendeten Bibliotheken importiert. Verwendet wurden klassiche Bibliotheken wie:
* pandas
* numpy
* plotly express
* sklearn

***

In [109]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

## Lesen der .csv

***

Nun wird die healthcare-dataset-stroke-data.csv mit Hilfe von pandas eingelesen.

***


In [110]:
# Read csv with pandas
data = pd.read_csv("healthcare-dataset-stroke-data.csv")
data

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


## Aufraeumen des DataFrames

***

Das DataFrame ist noch ziemlich unordentlich und wird im naechsten Abschnitt aufgeraumt.
Zusammenfassung der Schritte:
* id Spalte entfernen
* Alle nutzlosen Reihen entfernen (kein bmi Wert)
* ever_married zu 1 und 0
* Residence_type zu 1 und 0
* aufteilen der Spalten gender, work_type und smoking_status zu One Hot Encoded
* ersetzen von True und False zu 1 und 0

***

In [111]:
# Clean data (remove id, drop cells with bmi NaN)
data = data.drop(columns=["id"]) # remove id irrelevant
data = data.dropna(subset=["bmi"]) # drop columns with no bmi value
data["ever_married"] = data["ever_married"].map({"Yes": 1, "No": 0}) # change Yes and No to 1 and 0
data["Residence_type"] = data["Residence_type"].map({"Urban": 1, "Rural": 0}) # change Urban and Rural to 1 and 0
data = pd.get_dummies(data, columns=["gender", "work_type", "smoking_status"]) # one hot encode columns with more than 2 possibilities
data = data.replace({"True": 1, "False": 0}) # change True and False to 1 and 0

# move stroke to the end
move_stroke = data.pop("stroke")
data["stroke"] = move_stroke

# show cleaned data
data

,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,gender_Female,gender_Male,gender_Other,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes,stroke
0,67.0,0,1,1,1,228.69,36.6,False,True,False,False,False,True,False,False,False,True,False,False,1
2,80.0,0,1,1,0,105.92,32.5,False,True,False,False,False,True,False,False,False,False,True,False,1
3,49.0,0,0,1,1,171.23,34.4,True,False,False,False,False,True,False,False,False,False,False,True,1
4,79.0,1,0,1,0,174.12,24.0,True,False,False,False,False,False,True,False,False,False,True,False,1
5,81.0,0,0,1,1,186.21,29.0,False,True,False,False,False,True,False,False,False,True,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5104,13.0,0,0,0,0,103.08,18.6,True,False,False,False,False,False,False,True,True,False,False,False,0
5106,81.0,0,0,1,1,125.20,40.0,True,False,False,False,False,False,True,False,False,False,True,False,0
5107,35.0,0,0,1,0,82.99,30.6,True,False,False,False,False,False,True,False,False,False,True,False,0
5108,51.0,0,0,1,0,166.29,25.6,False,True,False,False,False,True,False,False,False,True,False,False,0


## Definieren der Trainingsdaten

***

Es werden alle vorhandenen Daten fuer x verwendet ausser Stroke, da Stroke die Zielklassifikation ist.
Anschliessend wird mit Hilfe von train_test_split von sk_learn die Daten aufgeteilt.

***

In [112]:
# split values into x and y
x = data.drop("stroke", axis=1)
y = data["stroke"]

# make train test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

## Definieren und fitten des Models

***

Wir benutzen einen **RandomForestClassifier**, Teil der Entscheidungsbaeume von sklearn und fitten unsere Trainingsdaten.

***

In [113]:
# fit the model
rfc = RandomForestClassifier(n_estimators=30) # through testing
rfc = rfc.fit(x_train, y_train)

## Vorhersagen der Testdaten und Auswertung

***

Nun wird der RandomForestClassifier auf die Testdaten angewendet und anschliessend mit der Fehlermetrik von sklearn bewertet

***

In [114]:
# predict and evaluate model on test split
predictions = rfc.predict(x_test)
accuracy = accuracy_score(y_test, predictions)
accuracy

0.9714867617107943

## Wichtige Informationen der Entscheidung herausfinden

***

Natuerlich wollen wir die Entscheidung des Baums zurueckfuehren und verstehen auf welche Feature am meisten geachtet wurde und das Ergebnis sind age, avg_glucose_level und bmi.

Also laesst sich kurz gesagt ein Grossteil der Entscheidung auf diese drei Feature zurueckfuehren und man koennte aehnlich gute Ergebnisse nur mit diesen Featuren erreichen.

***

In [118]:
# map feature names and their importances
feature_names = rfc.feature_names_in_
feature_importances = rfc.feature_importances_
for n, i in zip(feature_names, feature_importances):
    print(f"{round(i*100, 2)}%\t{n}")

23.6%	age
2.64%	hypertension
3.23%	heart_disease
1.64%	ever_married
3.3%	Residence_type
27.19%	avg_glucose_level
21.68%	bmi
2.24%	gender_Female
1.93%	gender_Male
0.0%	gender_Other
1.52%	work_type_Govt_job
0.0%	work_type_Never_worked
1.87%	work_type_Private
2.2%	work_type_Self-employed
0.08%	work_type_children
1.47%	smoking_status_Unknown
1.79%	smoking_status_formerly smoked
1.94%	smoking_status_never smoked
1.66%	smoking_status_smokes


## Plotten der drei wichtigsten Feature

***

Letztendlich plotten wir einen 3D Scatter von Plotly Express um die drei wichtigsten Feature zu veranschaulichen, alle gelben Baelle sind Daten die letztendlich auch einen *stroke* hatten und blaue die ohne.

Es erschliesst sich, dass letztendlich nur das Alter eine wirklich wichtige Rolle spielt und die Korrelation zwischen bmi, avg_glucose_level und age weniger relevant sind als das pure Alter.

***

In [119]:
# scatter plot of top 3 most important features
fig = px.scatter_3d(data[['age', 'avg_glucose_level', 'bmi']], 
                    x='age', y='avg_glucose_level', z='bmi', 
                    color=data["stroke"])
fig.show()